In [3]:
#!pip install opencv-contrib-python 
#!pip install opencv-python --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.6/32.6 MB 31.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.68
    Uninstalling opencv-python-4.7.0.68:
      Successfully uninstalled opencv-python-4.7.0.68


In [7]:
from abc import ABC, abstractmethod
import itertools
import os
import cv2 
import numpy as np
from scipy import signal

import matplotlib.pyplot as plt



# 3. Naive Approach to Fleet Jepson 

Given: 

$$Video = f(x,y,t)$$
$$ 3D Spatio-Temporal Gabor filter = g(x,y,t)$$

Filter response: 

$$s(x,y,t) = f(x,y,t)*g(x,y,t) = \rho(x,y,t) exp[j\phi(x,y,t)]$$

Now, according to fleep jepson $\phi(x,y,t)$ (phase signal) tracks the optical flow in a very stable manner. They derive OFCE as: 

$$ \phi_{x}a_{x} + \phi_{y}a_{y} + \phi_{t} = 0 ..... (1)$$ 
Here: $a = [\begin{array}{c} a_{x} \\ a_{y} \end{array}] = [\begin{array}{c} \dot x(x,y,t)  \\ \dot y(x,y,t) \end{array}]$, and $x = [\begin{array}{c} x \\ y \end{array}]$

Furthermore, they introduce the smoothness constraint: 

$$ a = \alpha n(x,y,t) = \alpha \frac{\nabla \phi(x,y,t)}{\lvert{\nabla \phi(x,y,t)}\rvert} ....(2)$$


### 1. Estimate of $a$ for the flow: 

use the above equation and constraints, and solving them together: 

as per slides, the smoothness constraint have gradient in $x,y$, i.e. $\nabla_{2} \phi(x,y,t) = [\begin{array}{c} \phi_{x} \\ \phi_{y} \end{array}]$ 

We substitute the component of $a$ in the above equation, and solve for $\alpha$: 

$$ a = \alpha \frac{\left[ \begin{array}{c} \dfrac{\partial f}{\partial x}(\phi(x,y,t)) \\ \dfrac{\partial f}{\partial y}(\phi(x,y,t)) \end{array}\right]}{\lvert{\nabla_2 \phi(x,y,t)}\rvert} = \left[\begin{array}{c} a_x \\ a_y \end{array}\right] $$

subsituting the above in the first equation, we get:

$$ \phi_x \left[\begin{array}{c} \alpha \dfrac{\dfrac{\partial \phi}{\partial x}}{\lvert \nabla_{2} \phi \rvert} \end{array}\right] \
+ \phi_y \left[\begin{array}{c} \alpha \dfrac{\dfrac{\partial \phi}{\partial y}}{\lvert \nabla_{2} \phi \rvert} \end{array}\right] \ 
+ \phi_t = 0 $$

$$=> \alpha \left[\dfrac{\phi_{x}^2}{\lvert \nabla_{2} \phi \rvert} + \dfrac{\phi_{y}^2}{\lvert \nabla_{2} \phi \rvert} \right] \
+ \phi_t =0 $$ 

$$\because \lvert\nabla_{2} \rvert^2 \phi = \phi_{x}^2 + \phi_{y}^2 $$
$$ => \alpha = \dfrac{- \phi_{t}}{\lvert \nabla_{2} \phi \rvert ^2} \lvert \nabla_{2} \phi \rvert $$

putting the $\alpha$ back to smoothing constraint, we get: 

####  $$ \hat a = \dfrac{- \phi_{t}}{\lvert \nabla_{2} \phi \rvert} \dfrac{\nabla_{2} \phi}{\lvert \nabla_{2} \phi \rvert} $$


### 2. Writing $\nabla \phi(x,y,t)$ in terms of $s(x,y,t)$

Give: 

$$ s(x,y,t) = \rho(x,y,t) exp[j\phi(x,y,t)]$$

$$ => \nabla_{3}\phi(x,y,t) = \left[\begin{array}{c} \dfrac{\partial \phi}{\partial x} \\ \dfrac{\partial \phi}{\partial y} \\ \dfrac{\partial \phi}{\partial t} \end{array} \right] = \left[\begin{array}{c} \dfrac{\partial}{\partial x} (\dfrac{\log(\dfrac{s}{\rho})}{j}) \\ \dfrac{\partial }{\partial y}(\dfrac{\log(\dfrac{s}{\rho})}{j}) \\ \dfrac{\partial}{\partial t}(\dfrac{\log(\dfrac{s}{\rho})}{j}) \end{array} \right] $$

$$=> \nabla_{3}\phi(x,y,t) = \left[\begin{array}{c} (\dfrac{-j}{s}) (s_{x}) \\ (\dfrac{-j}{s}) (s_{y}) \\ (\dfrac{-j}{s}) (s_{t}) \end{array} \right] - \left[\begin{array}{c} (\dfrac{-j}{\rho}) (\rho_{x}) \\ (\dfrac{-j}{\rho}) (\rho_{y}) \\ (\dfrac{-j}{\rho}) (\rho_{t}) \end{array} \right] $$

$$=> \nabla_{3}\phi(x,y,t) = -j [\dfrac{\nabla_{3} s}{s} - \dfrac{\nabla_{3} \rho}{\rho} ]$$
$$=> \nabla_{3}\phi(x,y,t) = -j [\dfrac{s^* \nabla_{3} s}{s *  s^*} - \dfrac{\rho^* \nabla_{3} \rho}{\rho *  \rho^*} ]$$
$$ \because magnitude of s is equal to \rho, i.e. \lvert s \rvert = \lvert \rho \rvert $$
$$also, \rho does not have any complex conjugate$$
$$=> \nabla_{3}\phi(x,y,t) = -j [\dfrac{s^* \nabla_{3} s}{\lvert s \rvert^2} - \dfrac{\rho^* \nabla_{3} \rho}{\lvert s \rvert^2}]$$
$$=> \nabla_{3}\phi(x,y,t) = \dfrac{-j}{{\lvert s \rvert^2}} [s^* \nabla_{3} s - \rho^* \nabla_{3} \rho]$$
$$=> \nabla_{3}\phi(x,y,t) = \dfrac{-j}{{\lvert s \rvert^2}} [s^* \nabla_{3} s - \rho \nabla_{3} \rho]$$

Now, expanding to check the term $s^*\nabla s$ 
$$ s^* \nabla_{3} s = \rho exp(-j\phi) [\nabla \rho exp(j\phi) + j\rho \nabla \phi exp(j\phi)]$$
$$ =  [\rho \nabla \rho + j\rho^2 \nabla \phi] $$

Thus, substracting the terms $\rho \nabla \rho$ from the term $s^* \nabla s$ basically gives the iota times imaginary part of it : $j \rho^2 \nabla \phi$

$$=> \nabla_{3}\phi(x,y,t) = \dfrac{-j}{\lvert s \rvert^2} [j \rho^2 \nabla \phi] $$
$$=> \nabla_{3}\phi(x,y,t) = \dfrac{Im(s^* \nabla s)}{\lvert s \rvert^2} $$

Hence Proved. 





In [15]:
# NOTE: alpha is kept between 0 and 1 in practical applications.

def spatiotemporal_gabor_filter(freq_space, freq_time, theta, sigma_space, sigma_time, aspect_ratio):
    """
    Define a 3D spatiotemporal Gabor filter.

    Parameters:
        freq_space (float): Spatial frequency in cycles per pixel.
        freq_time (float): Temporal frequency in cycles per frame.
        theta (float): Orientation of the Gabor filter in degrees.
        sigma_space (float): Spatial standard deviation in pixels.
        sigma_time (float): Temporal standard deviation in frames.
        aspect_ratio (float): Aspect ratio of the Gabor filter (ratio of the spatial standard deviation along the major axis to that along the minor axis).

    Returns:
        filter (ndarray): 3D spatiotemporal Gabor filter.
    """
    # Convert angle to radians
    theta = np.deg2rad(theta)

    # Define spatial and temporal grids
    x = np.arange(-10*sigma_space, 10*sigma_space+1) # 3D grid in space
    y = np.arange(-10*sigma_space, 10*sigma_space+1) # 3D grid in space
    t = np.arange(-10*sigma_time, 10*sigma_time+1)   # 3D grid in time
    x, y, t = np.meshgrid(x, y, t, indexing='ij')

    # Calculate rotated coordinates
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)

    # Define spatiotemporal Gabor function
    spatial_term = np.exp(-(x_theta**2 + aspect_ratio**2 * y_theta**2) / (2 * sigma_space**2))
    temporal_term = np.exp(-(t**2) / (2 * sigma_time**2)) * np.cos(2 * np.pi * freq_time * t)
    gabor = spatial_term * temporal_term

    # Normalize filter
    filter = gabor / np.sum(gabor)

    return filter

class OpticalFlowEstimator:
    
    def __init__(self, video_path, width=1920, height=1080, is_yuv=True, ksize=31, sigma=5, theta=np.pi/4, lamda=np.pi/4, gamma=0.5, phi=0, alpha=0.1):
        self.video_path = video_path
        self.width = width
        self.height = height
        self.ksize = ksize
        self.sigma = sigma
        self.theta = theta
        self.lamda = lamda
        self.gamma = gamma
        self.phi = phi
        self.alpha = alpha
        self.prev_gray = None
        self.kernel = None
        self.is_yuv = is_yuv

    def _vid_read(self):
        if self.is_yuv:
            self.frames = self.load_vid_to_frames()
        else:
            self.cap = cv2.VideoCapture(self.video_path)

    #read the yuv420 file video
    def load_vid_to_frames(self):
        """
        :param filename: the path to the yuv file
        :param height: the height of the video
        :param weight: the weight of the video
        :return: list of frames (flaot32)
        """
        #read the yuv file
        with open(self.video_path, 'rb') as f:
            yuv = f.read()
        
        #get the frame size
        frame_size = self.width * self.height * 3 // 2 #yuv420
        
        #get the number of frames
        num_frames = len(yuv) // frame_size
        
        #convert the yuv file to frames
        frames = []
        for i in range(num_frames):
            #get the frame
            frame = yuv[i*frame_size:(i+1)*frame_size]
            
            #convert the frame to rgb
            frame = np.frombuffer(frame, dtype=np.uint8)
            frame = frame.reshape((self.height*3//2, self.width))
            frame = cv2.cvtColor(frame, cv2.COLOR_YUV2RGB_I420)
            
            #add the frame to the list
            frames.append(frame)
        
        return frames
    
    def _create_gabor_kernel(self):
        self.kernel = cv2.getGaborKernel((self.ksize, self.ksize), self.sigma, self.theta, self.lamda, self.gamma, self.phi, ktype=cv2.CV_32F)
        self.kernel = np.repeat(self.kernel[:, :, np.newaxis], 3, axis=2)
    
    def _compute_optical_flow(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply Gabor filter to current and previous frames
        filtered = cv2.filter2D(np.concatenate((self.prev_gray[:, :, np.newaxis], gray[:, :, np.newaxis]), axis=2), -1, self.kernel)
        prev_filtered, filtered = np.split(filtered, 2, axis=2)

        # Compute phase signal and gradients
        phase = np.arctan2(np.sin(filtered - prev_filtered), np.cos(filtered - prev_filtered))
        grad_x, grad_y, grad_t = np.gradient(phase)

        # Compute optical flow using Equation (1) and (2)
        norm = np.sqrt(grad_x**2 + grad_y**2)
        norm[norm == 0] = 1e-5  # Avoid division by zero
        a_x = -grad_x/norm
        a_y = -grad_y/norm
        a_t = grad_t/norm
        a_x[np.isnan(a_x)] = 0
        a_y[np.isnan(a_y)] = 0
        a_t[np.isnan(a_t)] = 0
        a_x[np.isinf(a_x)] = 0
        a_y[np.isinf(a_y)] = 0
        a_t[np.isinf(a_t)] = 0
        a = np.stack((a_x, a_y), axis=2)
        n = np.stack((grad_x, grad_y), axis=2)
        n_norm = np.sqrt(n[:, :, 0]**2 + n[:, :, 1]**2)
        n_norm[n_norm == 0] = 1e-5  # Avoid division by zero
        n[:, :, 0] /= n_norm
        n[:, :, 1] /= n_norm
        a = self.alpha*n
        
        self.prev_gray = gray
        
        return a
        
    def run(self):
        self._create_gabor_kernel()
        
        while True:
            # Read next frame and check if it was successfully read
            if self.is_yuv:
                frame = self.frames.pop(0)
            else:
                ret, frame = self.cap.read()
                if not ret:
                    break
                    
            # Compute optical flow
            flow = self._compute_optical_flow(frame)
            
            # Display optical flow
            flow += 0.5  # Shift to [0, 1] range for visualization
            #using plt 
            plt.imshow(flow)
            plt.show()
                    
if __name__ == "__main__":
    # Create optical flow estimator
    estimator = OpticalFlowEstimator("./Beauty_1920x1080_120fps_420_8bit_YUV_RAW/Beauty_1920x1080_120fps_420_8bit_YUV.yuv")
    #read vids 
    estimator._vid_read()
    # Run optical flow estimator
    estimator.run()
    


TypeError: 'NoneType' object is not subscriptable